In [1]:
import  pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
customer=pd.read_csv(r"C:\Users\Shiv_Shakti\Desktop\PG CERT DS\Term 2\0CASE STUDY\Case Study 1 - Retail Case Study\Customer.csv")
prod_info=pd.read_csv(r"C:\Users\Shiv_Shakti\Desktop\PG CERT DS\Term 2\0CASE STUDY\Case Study 1 - Retail Case Study\prod_cat_info.csv")
transaction=pd.read_csv(r"C:\Users\Shiv_Shakti\Desktop\PG CERT DS\Term 2\0CASE STUDY\Case Study 1 - Retail Case Study\Transactions.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Shiv_Shakti\\Desktop\\PG CERT DS\\Term 2\\0CASE STUDY\\Case Study 1 - Retail Case Study\\Customer.csv'

In [ ]:
customer.head(3)

In [ ]:
prod_info.head(3)

In [ ]:
#renaming "prod_sub_cat_code" column in prod_info table to make it similar to transaction table.
prod_info.rename(columns={"prod_sub_cat_code":"prod_subcat_code"}, inplace=True)

In [ ]:
transaction.head(3)

1. Merge the datasets Customers, Product Hierarchy and Transactions as Customer_Final. Ensure to
keep all customers who have done transactions with us and select the join type accordingly

In [ ]:
#merge transaction and product info table to create a new table "prod_concat"
prod_concat= pd.merge(left=transaction, right=prod_info, on=["prod_subcat_code","prod_cat_code"], how="left")

In [ ]:
#merge "prod_concat" and "customer" table and create the final table "customer_final".
customer_final=pd.merge(left=prod_concat,right=customer, right_on="customer_Id", left_on="cust_id", how="left")

In [ ]:
customer_final.head()

In [ ]:
customer_final.dtypes

In [ ]:
# converting "DOB" and "tran_date" from object dtype to dates
customer_final["DOB"] = pd.to_datetime(customer_final["DOB"], format="%d-%m-%Y")

In [ ]:
customer_final['DOB'].head(10)

In [ ]:
customer_final["tran_date"] = pd.to_datetime(customer_final["tran_date"])

In [ ]:
customer_final["tran_date"].head(10)

In [ ]:
customer_final.duplicated().sum()

In [ ]:
# dropping duplicate rows
customer_final.drop_duplicates(inplace=True)

In [ ]:
customer_final.duplicated().sum()

2(a) Get the column names and their corresponding data types

In [ ]:
column_info = customer_final.dtypes
print(column_info)

 b. Top/Bottom 10 observations

In [ ]:
# top 10 observations
customer_final.head(10)

In [ ]:
#bottom 10 observations
customer_final.tail(10)

 c. “Five-number summary” for continuous variables (min, Q1, median, Q3 and max)


In [ ]:
customer_final.describe().drop(['count','mean','std'])

D.Frequency tables for all the categorical variables

In [ ]:
customer_final.describe(include='object')

In [ ]:
sns.histplot(customer_final.prod_cat, kde=True)
plt.show()
sns.histplot(customer_final.prod_subcat, kde=True)
plt.show()
sns.histplot(customer_final.Gender)
plt.show()
sns.histplot(customer_final.Store_type, kde=True)
plt.show()

3. Generate histograms for all continuous variables and frequency bars for categorical variables

In [ ]:
conti_customer = customer_final.loc[:,['prod_subcat_code','prod_cat_code', 'Qty', 'Rate', 'Tax', 'total_amt']]

In [ ]:
conti_customer.columns

Histogram of all continuous variables

In [ ]:
for var in conti_customer.columns:
    conti_customer[var].plot(kind='hist')
    plt.title(var)
    plt.show()

Bar chart of categorical variables

In [ ]:
category_customer = customer_final.loc[:,customer_final.dtypes=='object']

In [ ]:
category_customer.head()

In [ ]:
fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(2, 2, 1)
sns.countplot(data=category_customer, x='Gender', ax=ax1)

ax2 = fig.add_subplot(2, 2, 2)
sns.countplot(data=category_customer, x='Store_type', ax=ax2)
ax2.set_xlabel('Store Type')

ax3 = fig.add_subplot(2, 2, 3)
sns.countplot(data=category_customer, x='prod_cat', ax=ax3)
ax3.set_xlabel('Product Category')
ax3.set_xticklabels(ax3.get_xticklabels(), rotation=90)

ax4 = fig.add_subplot(2, 2, 4)
category_customer['prod_subcat'].value_counts().plot(kind='barh', ax=ax4)
ax4.set_xlabel('Count')
ax4.set_ylabel('Product Subcategory')

plt.subplots_adjust(wspace=1)
plt.show()


 4. Calculate the following information using the merged dataset :

 a. Time period of the available transaction data

In [ ]:
customer_final.sort_values(by="tran_date")

In [ ]:
min_date = customer_final["tran_date"].min()
min_date

In [ ]:
max_date = customer_final["tran_date"].max()
max_date

In [ ]:
print("Time period of the available transaction data is from "+ pd.Timestamp.strftime(min_date,format="%d-%m-%Y") + " to " + pd.Timestamp.strftime(max_date,format="%d-%m-%Y"))

(b) Count of transactions where the total amount of transaction was negative

In [ ]:
customer_final.head()

In [ ]:
#count of transaction_ids where total_amt was negative
negative_transaction = customer_final.loc[customer_final["total_amt"] < 0,"transaction_id"].count()
print("Count of transactions where the total amount of transaction was negative is",negative_transaction)

(5) Analyze which product categories are more popular among females vs male customers

In [ ]:
#groupby the data set on the basis of "Gender" and "prod_cat"
product_gender = customer_final.groupby(["Gender","prod_cat"])[["Qty"]].sum().reset_index()
product_gender

In [ ]:
#converting to pivot table for better view
product_gender.pivot(index="Gender",columns="prod_cat",values="Qty")

(6) Which City code has the maximum customers and what was the percentage of customers from that city?

In [ ]:
customer_group = customer_final.groupby('city_code')['customer_Id'].count().sort_values(ascending =False)
customer_group

In [ ]:
percentage = round((customer_group[4.0] / customer_group.sum()) * 100,2)
print("City code 4.0 has the maximum customers and the percentage of customers from that city is ",percentage)

(7) Which store type sells the maximum products by value and by quantity?

In [ ]:
customer_final.groupby("Store_type")[["Qty","Rate"]].sum().sort_values(by="Qty",ascending=False).head(1)

In [ ]:
print('e-Shop store sell the maximum products by value and by quantity')

(8) What was the total amount earned from the "Electronics" and "Clothing" categories from Flagship Stores?

In [ ]:
store_group = round(customer_final.pivot_table(index = "prod_cat",columns="Store_type", values="total_amt", aggfunc='sum'),2)
store_group

In [ ]:
store_group.loc[["Clothing","Electronics"],"Flagship store"].sum()

(9) What was the total amount earned from "Male" customers under the "Electronics" category?

In [ ]:
gender_group = round(customer_final.pivot_table(index = "prod_cat",columns="Gender", values="total_amt", aggfunc='sum'),2)
gender_group

In [ ]:
male_earning = gender_group.loc["Electronics","M"]
print("The total amount earned from Male customers under the Electronics category is",male_earning)

(10) How many customers have more than 10 unique transactions, after removing all transactions which have any negative amounts?

In [ ]:
#creating a new dataframe that does not contain transactions with negative values
pos_trans = customer_final.loc[customer_final["total_amt"]>0,:]
pos_trans

In [ ]:
# creating a dataframe that contains unique transactions 
unique_trans = pos_trans.groupby(['customer_Id','prod_cat','prod_subcat'])['transaction_id'].count().reset_index()
unique_trans

In [ ]:
# now finding the customers which have unique transactions greater than 10
unique_trans_count = unique_trans.groupby('customer_Id')['transaction_id'].count().reset_index()
unique_trans_count

In [ ]:
unique_trans_count[unique_trans_count['transaction_id'] > 10]

In [ ]:
print('There are no unique transactions greater than 10')

(11) For all customers aged between 25-35, find out:

a) What was the total amount spent for 'Electronics' and 'Books' product categories?

In [ ]:
# Getting Ages for customers from DOB to 1st Mar, 2014.
customer_final['DOB']=pd.to_datetime(customer_final['DOB'])
dateforage=pd.to_datetime('2014-3-1')
customer_final['Age']=((dateforage - customer_final['DOB']).dt.total_seconds()/(365.25*24*3600)).astype('int')

In [ ]:
# Filtering by ages 25 to 35.
customer_final_25_35=customer_final.loc[(customer_final.Age > 25)&(customer_final.Age <35)]
# a. What was the total amount spent for “Electronics” and “Books” product categories?
customer_final_25_35_EB=customer_final_25_35.loc[(customer_final_25_35.prod_cat =='Electronics')|(customer_final_25_35.prod_cat =='Books')]
display(customer_final_25_35_EB.agg({'total_amt':'sum'}).reset_index())

(b) What was the total amount spent by these customers between 1st Jan 2014 to 1st Mar 2014?

In [ ]:
filter = ((customer_final['tran_date'] >= '1st Jan, 2014')&(customer_final['tran_date'] <= '1st Mar, 2014'))
display(customer_final['total_amt'].loc[filter].sum())